In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from depsurf import Versions
from depsurf.output import (
    get_legend_handles_labels,
    label_multiline_text,
    plot_yticks,
    save_fig,
    load_df,
    plot_bar,
)

captions = {
    Versions.REGULAR: "Linux Kernel Version",
    Versions.ARCH: "Arch for Linux 5.4",
    Versions.FLAVOR: "Flavor for Linux 5.4",
}


groups = captions.keys()


def plot_fig(df_all: pd.DataFrame, ylabel, columns=None, lines_fn=None, ylim_mult=1.1):
    df_all = df_all.drop(
        index=(
            Versions.FLAVOR,
            next(v for v in Versions.FLAVOR if v.flavor == "oracle"),
        )
    )

    fig, axs = plt.subplots(
        1,
        3,
        figsize=(12, 2.25),
        sharey=True,
        width_ratios=[len(df_all.loc[group]) for group in groups],
        gridspec_kw={"wspace": 0.05, "hspace": 0.075},
    )
    axs: list[plt.Axes]

    columns = columns if columns is not None else df_all.columns
    ylim = df_all[columns].sum(axis=1).max() * ylim_mult
    for ax, group in zip(axs, groups):

        df = df_all.loc[group]
        plot_bar(ax, df, columns)

        xs = np.arange(len(df))
        ax.tick_params(axis="both", labelsize=9)
        ax.set_xticks(xs, [group.version_to_str(v) for v in df.index], rotation=0)
        ax.set_xlabel(captions[group])
        ax.set_xlim(-0.5, len(df) - 0.5)

        ax.set_ylim(0, ylim)

        if lines_fn is None:
            continue

        with plt.rc_context({"font.family": "monospace"}):
            for i, (index, row) in enumerate(df.iterrows()):
                lines, colors = zip(*lines_fn(row))
                label_multiline_text(ax, i, ylim, lines, colors=colors, fontsize=9)

    plot_yticks(axs[0])
    axs[0].set_ylabel(ylabel)

    handles, labels = get_legend_handles_labels(fig)
    fig.legend(handles, labels, loc="upper center", ncol=5, bbox_to_anchor=(0.5, 1.02))
    return fig

In [3]:
from depsurf import RenameType


def gcc_short_ver(s):
    return s[: s.rfind(".")]


df_all = load_df("rename")
fig = plot_fig(
    df_all,
    columns=list(RenameType),
    ylabel="Number of Funcions Renamed",
    lines_fn=lambda row: [
        (gcc_short_ver(row["GCC"]), "black"),
        (f'{row["Renamed"] / row["Total"]:.0%}', "black"),
    ],
    ylim_mult=1.2,
)
save_fig(fig, "rename", close=True)

[           pd.py:53 ] INFO: Loaded df from /Users/szhong/Downloads/bpf-study/output/rename.pkl


[          mpl.py:79 ] INFO: Saved figure to /Users/szhong/Downloads/bpf-study/paper/figs/rename.pdf


In [4]:
from depsurf import InlineType

fig = plot_fig(
    load_df("inline"),
    lines_fn=lambda row: [(f"{row[InlineType.PARTIAL]/1000:.1f}k", "darkred")],
    ylabel="Number of Functions",
    ylim_mult=1.1,
)
save_fig(fig, "inline", close=True)

[           pd.py:53 ] INFO: Loaded df from /Users/szhong/Downloads/bpf-study/output/inline.pkl
[          mpl.py:79 ] INFO: Saved figure to /Users/szhong/Downloads/bpf-study/paper/figs/inline.pdf


In [5]:
from depsurf import CollisionType

fig = plot_fig(
    load_df("dup"),
    columns=[
        CollisionType.INCLUDE_DUP,
        CollisionType.STATIC_STATIC,
        CollisionType.STATIC_GLOBAL,
    ],
    lines_fn=lambda row: [
        (f"{row[CollisionType.STATIC_GLOBAL]}", "darkgreen"),
        (f"{row[CollisionType.STATIC_STATIC]}", "darkred"),
    ],
    ylabel="Number of Cases",
    ylim_mult=1.2,
)
save_fig(fig, "dup", close=True)

[           pd.py:53 ] INFO: Loaded df from /Users/szhong/Downloads/bpf-study/output/dup.pkl
[          mpl.py:79 ] INFO: Saved figure to /Users/szhong/Downloads/bpf-study/paper/figs/dup.pdf
